In [24]:
# sudo service mysql stop
# docker run --name mysql_call -p 3306:3306 -e MYSQL_ROOT_PASSWORD=your_secret_pwd -d mysql:latest

In [25]:
import pymysql

In [26]:
with open('mysql.pass','r') as file:
    mysql_pass = file.read().replace('\n', '')
    file.close()

In [27]:
conn = pymysql.connect(
            host = '127.0.0.1', 
            user = 'root',
            passwd = mysql_pass,
            db = 'mysql',
            autocommit = True
        )

In [28]:
cursor = conn.cursor()

In [29]:
query = "create database testdb;"
cursor.execute(query)

1

In [30]:
query = "use testdb;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [31]:
query = "show tables;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [32]:
query = "drop database testdb;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)